In [1]:
import serial
import time
import binascii
import struct


try:
    s = serial.Serial('COM5', baudrate=115200, timeout=0.1)
    
except serial.SerialException:
    s.close()
    s = serial.Serial('COM3', baudrate=115200, timeout=0.1)
    pass
    print('eeeror')


gripper_open =  1
global is_open
is_open = 1
position_gripper = 100
s.write(bytes(str(gripper_open) + "," + str(position_gripper)+ "\n",'utf-8'))


def open_gripper():
    global is_open
    open_gripper =  1
    position_gripper = 100
    if is_open==0:
        s.write(bytes(str(open_gripper) + "," + str(position_gripper)+ "\n",'utf-8'))
        print("opening the gripper")
#     time.sleep(1)
        s.flushInput()
        is_open = True
        time.sleep(1)
def close_gripper():
    global is_open
    open_gripper =  2
    position_gripper = 100
    if is_open==1:
        s.write(bytes(str(open_gripper) + "," + str(position_gripper)+ "\n",'utf-8'))
        time.sleep(1)
        is_open = 0
        print("closing the gripper")
    
        s.flushInput()
    
    
    

In [3]:
open_gripper()


opening the gripper


In [2]:
close_gripper()

closing the gripper


In [ ]:
gripper_open =  'hola'
position_gripper = 100
s.write(bytes(str(gripper_open) + "," + str(position_gripper)+ "\n",'utf-8'))

In [ ]:
import mediapipe as mp
import cv2
import numpy as np
from IPython.display import clear_output

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
distance_open = 200

with mp_hands.Hands(min_detection_confidence = 0.2, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        clear_output(wait=True)
        ret, image = cap.read()
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        

        if results.multi_hand_landmarks:
            
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS,
                                          # mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                          # mp_drawing.DrawingSpec(color=(250,44,250), thickness=2, circle_radius=4),
                                         )
                
                finger_point = np.array((hand.landmark[8].x *image.shape[1], hand.landmark[8].y*image.shape[0]))
                thumb = np.array((hand.landmark[4].x *image.shape[1], hand.landmark[4].y*image.shape[0]))
                distance_fingers = np.linalg.norm(finger_point - thumb)
                print("finger_point: ", finger_point)
                print("distance: ", distance_fingers)
                
                if distance_fingers <= distance_open:
                    close_gripper()
                else:
                    open_gripper()
                    
                    
                    
                
                                         
        cv2.imshow("HT", image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
                break
cap.release()
cv2.destroyAllWindows()